# Jeopardy dataset manipulation with Pandas

#### Project Goals

Write several functions that investigate a dataset of _Jeopardy!_ questions and answers.

#### Author:

Oleg Astafyev

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

# Loading csv into the 'jeopardy' data frame
jeopardy = pd.read_csv('jeopardy.csv')

print(jeopardy.columns)

# Renaming columns (removing leading whitespace)
jeopardy.rename(columns=lambda x: x.lstrip(), inplace=True)
print(jeopardy.columns)

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')
Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')


2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [2]:
# Function that filters the dataset for questions that contains all of the words in a list of words
# Returns a filtered dataset dataframe

def filter_for(wordlist):

  # Preventing capitalization from affecting the filtering by making all words in the wordlist lowercase
  wordlist = [word.lower() for word in wordlist]
    
  result_df = jeopardy[jeopardy['Question'].apply(lambda x: set(wordlist).issubset(set(x.lower().split())))]

  return result_df

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [3]:
filtered1 = filter_for(['king','england'])

print(filtered1['Question'])

6337               In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                             This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
13454                                                It's the number that followed the last king of England named William
14912                   This country's King Louis IV was nicknamed "Louis From Overseas" because he was raised in England
18076                     In 1199 this crusader king of England was mortally wounded while besieging the castle of Chalus
                                                               ...                                                       
200369    8th C. King Offa built a 170-mile north-south dike that still partly marks the border of England & this country
201168                                           Popular Saint-Exupery character waiting around to become king of England
208742                  

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [4]:
# Converting string values to float in a new column
jeopardy['ValueFloat'] = jeopardy['Value'].apply(lambda x: float(0) if x == 'None' else float(x.lstrip('$').replace(',','')))

# Filtering the dataset only for questions with the word king
filtered2 = filter_for(['king'])

print('The average value of questions containing the word \"King\" is:')
print(filtered2['ValueFloat'].mean())

The average value of questions containing the word "King" is:
805.4698795180723


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [5]:
# Function that returns the count of unique answers to all of the questions in a dataset

def count_uniq(dataframe):
    
    return dataframe['Answer'].nunique()

print("There are " + str(count_uniq(jeopardy)) + " unique answers in the whole Jeopardy dataset")

There are 88268 unique answers in the whole Jeopardy dataset


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?

In [6]:
# Adding the column with just the year information

jeopardy['Year'] = jeopardy['Air Date'].apply(lambda x: str(x[:4]))

# How many questions from the 90s use the word "Computer" compared to questions from the 2000s?

filtered3 = filter_for(['Computer'])

nineties = []
twothousands = []

filtered3['Year'].apply(lambda x: nineties.append(x) if int(x)<2000 else twothousands.append(x))

print(str(len(nineties)) + " questions from the 90s use the word \"Computer\", as opposed to " + str(len(twothousands)) + " questions from the 2000s")

71 questions from the 90s use the word "Computer", as opposed to 244 questions from the 2000s


* Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?

In [7]:
# Let's see how many occurrences of the "Literature" category are in each of the rounds
numLitQ1 = len(jeopardy[(jeopardy['Round'] == 'Jeopardy!') & (jeopardy['Category'] == 'LITERATURE')].count(axis=1))
numLitQ2 = len(jeopardy[(jeopardy['Round'] == 'Double Jeopardy!') & (jeopardy['Category'] == 'LITERATURE')].count(axis=1))
numLitQ3 = len(jeopardy[(jeopardy['Round'] == 'Final Jeopardy!') & (jeopardy['Category'] == 'LITERATURE')].count(axis=1))
numLitQ4 = len(jeopardy[(jeopardy['Round'] == 'Tiebreaker') & (jeopardy['Category'] == 'LITERATURE')].count(axis=1))

# A list with number of occurrences of the "Literature" category per round
numLit = [numLitQ1, numLitQ2, numLitQ3, numLitQ4]

if numLit.index(max(numLit)) == 0:
    
    print("The category \"Literature\" is most likely to be found in \"Jeopardy!\" round")
    
elif numLit.index(max(numLit)) == 1:
    
    print("The category \"Literature\" is most likely to be found in \"Double Jeopardy!\" round")

elif numLit.index(max(numLit)) == 2:
    
    print("The category \"Literature\" is most likely to be found in \"Final Jeopardy!\" round")

elif numLit.index(max(numLit)) == 3:
    
    print("The category \"Literature\" is most likely to be found in \"Tiebreaker\" round")


The category "Literature" is most likely to be found in "Double Jeopardy!" round


* Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [ ]:
print("Welcome to Jeopardy!")
print("")
print("Please choose the round you would like to play (type in the respective integer):" + "\n" + "1. Jeopardy!" + "\n" +"2. Double Jeopardy!" + "\n" + "3. Final Jeopardy!" + "\n" + "4. Tiebreaker")
print("")

# Variable to store user's input and chosen round name
answer = input()
rndName = ""

# Choosing the round
if answer == "1":
    rndName = "Jeopardy!"
    print("You've chosen Jeopardy!")
elif answer == "2":
    rndName = "Double Jeopardy!"
    print("You've chosen Double Jeopardy!")
elif answer == "3":
    rndName = "Final Jeopardy!"
    print("You've chosen Final Jeopardy!")
elif answer == "4":
    rndName = "Tiebreaker"
    print("You've chosen Tiebreaker")
else:
    print("Sorry, there is no round under that integer. Maybe try again?")
    
# 
print("")
print("Now get ready for your question! In 3,2,1...")
print("")

# Getting the question and answer from the whole jeopardy dataset in the required category

questionDf = jeopardy[jeopardy['Round'] == rndName].sample(n=1,replace=False) 
print(questionDf['Question'].values[0])

print(questionDf['Answer'].values[0])

# Prompting user for answer + ensuring its "comparison" compatibility
qAnswer = input()
qAnswer = qAnswer.strip().lower()

corrAnswer = questionDf['Answer'].values[0].lower()

if qAnswer == '':
    print("Sorry, you did not type anything in")
elif qAnswer == corrAnswer:
    print("Amazing! You got it right!")
else:
    print("Sorry, wrong answer :( ")

Welcome to Jeopardy!

Please choose the round you would like to play (type in the respective integer):
1. Jeopardy!
2. Double Jeopardy!
3. Final Jeopardy!
4. Tiebreaker

